<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/dev_branch_EDA_Prediction/ReHealthCo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Empfehlung Max Vorsichtshalber: Löschen aller Variablen
%reset -f

In [ ]:
# import libraries
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
from pandas_profiling import ProfileReport
# from functools import reduce - benötigt?

In [ ]:
# mount drive to access data
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Connect to source database
def create_connection(db_file):
  """ create a database connection to SQLite database"""
  conn = None
  try:
    conn = sqlite3.connect(db_file)
    return conn
  except Error as error:
    print("Error while connecting to sqlite", error)
  # finally:
  #   if conn:
  #     conn.close()

# Erstellung der (leeren) Datenbanken (asthma, covid, metabolic, rehealthco)

In [ ]:
material_path = "/content/drive/Shareddrives/Projektarbeit DMA Gruppe D"
# conn = create_connection() - verbindung zur Datenbank
# cur = conn.cursor() - objekt auf datenbank, das die SQL-Anfragen ausführen kann

In [ ]:
# create a temporary empty ast_database in the memory
ast_db = f"{material_path}/source_dbs/asthma.db"
ast_conn = create_connection(ast_db)
ast_cur = ast_conn.cursor()
print("Database connection is successfully")

In [ ]:
# create a temporary empty cov_database in the memory
cov_db = f"{material_path}/source_dbs/covid.db"
cov_conn = create_connection(cov_db)
cov_cur = cov_conn.cursor()
print("Database connection is successfully")

In [ ]:
# create a temporary empty met_database in the memory
met_db = f"{material_path}/source_dbs/metabolic.db"
met_conn = create_connection(met_db)
met_cur = met_conn.cursor()
print("Database connection is successfully")

In [ ]:
# create a temporary empty rhc_database in the memory
rhc_db = f"{material_path}/source_dbs/rehealthco.db"
rhc_conn = create_connection(rhc_db)
rhc_cur = rhc_conn.cursor()
print("Database connection is successfully")

# Erstellung der Tabellen

In [ ]:
path = "/content/drive/Shareddrives/Projektarbeit DMA Gruppe D/source_dbs/sql_create.txt"
myfile = ""
with open(path, "r") as f:
  myfile=f.read()
#  print(myfile)

In [ ]:
ast_cur.executescript(myfile)
ast_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in ast_database
# ast_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(ast_cur.fetchall())

In [ ]:
cov_cur.executescript(myfile)
cov_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in cov_database
# cov_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cov_cur.fetchall())

In [ ]:
met_cur.executescript(myfile)
met_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in met_database
# met_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(met_cur.fetchall())

In [ ]:
rhc_cur.executescript(myfile)
rhc_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in cov_database
# rhc_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(rhc_cur.fetchall())

# Datenbanken befüllen mit csv-Dateien

In [ ]:
csv_path_ast = f"{material_path}/csv_data/asthma"
csv_path_cov = f"{material_path}/csv_data/covid19"
csv_path_met = f"{material_path}/csv_data/metabolic_syndrome_disease"

In [ ]:
# load the data into a Pandas DataFrame
astpat_df = pd.read_csv(f"{csv_path_ast}/patients.csv")
astcon_df = pd.read_csv(f"{csv_path_ast}/conditions.csv")
astmed_df = pd.read_csv(f"{csv_path_ast}/medications.csv")
astpro_df = pd.read_csv(f"{csv_path_ast}/procedures.csv")
astobs_df = pd.read_csv(f"{csv_path_ast}/observations.csv")

covpat_df = pd.read_csv(f"{csv_path_cov}/patients.csv")
covcon_df = pd.read_csv(f"{csv_path_cov}/conditions.csv")
covmed_df = pd.read_csv(f"{csv_path_cov}/medications.csv")
covpro_df = pd.read_csv(f"{csv_path_cov}/procedures.csv")
covobs_df = pd.read_csv(f"{csv_path_cov}/observations.csv")

metpat_df = pd.read_csv(f"{csv_path_met}/patients.csv")
metcon_df = pd.read_csv(f"{csv_path_met}/conditions.csv")
metmed_df = pd.read_csv(f"{csv_path_met}/medications.csv")
metpro_df = pd.read_csv(f"{csv_path_met}/procedures.csv")
metobs_df = pd.read_csv(f"{csv_path_met}/observations.csv")

# write the data to a sqlite table
astpat_df.to_sql("patients", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astcon_df.to_sql("conditions", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astmed_df.to_sql("medications", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astpro_df.to_sql("procedures", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astobs_df.to_sql("observations", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

covpat_df.to_sql("patients", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covcon_df.to_sql("conditions", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covmed_df.to_sql("medications", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covpro_df.to_sql("procedures", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covobs_df.to_sql("observations", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

metpat_df.to_sql("patients", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metcon_df.to_sql("conditions", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metmed_df.to_sql("medications", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metpro_df.to_sql("procedures", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metobs_df.to_sql("observations", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

# Abschluss
ast_conn.commit()
cov_conn.commit()
met_conn.commit()

In [ ]:
# check dataframe asthma/covid19/metabolic
# print(astpat_df)
# print(astcon_df)
# print(astmed_df)
# print(astpro_df)
# print(astobs_df)

# print(covpat_df)
# print(covcon_df)
# print(covmed_df)
# print(covpro_df)
# print(covobs_df)

# print(metpat_df)
# print(metcon_df)
# print(metmed_df)
# print(metpro_df)
# print(metobs_df)

In [ ]:
# Check data in patients

# check data in a table astpat
# ast_cur.execute("SELECT * FROM patients;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covpat
# cov_cur.execute("SELECT * FROM patients;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metpat
# met_cur.execute("SELECT * FROM patients;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in conditions

# check data in a table astcon
# ast_cur.execute("SELECT * FROM conditions;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covcon
# cov_cur.execute("SELECT * FROM conditions;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metcon
# met_cur.execute("SELECT * FROM conditions;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in medications

# check data in a table astmed
# ast_cur.execute("SELECT * FROM medications;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covmed
# cov_cur.execute("SELECT * FROM medications;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metmed
# met_cur.execute("SELECT * FROM medications;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in procedures

# check data in a table astpro
# ast_cur.execute("SELECT * FROM procedures;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covpro
# cov_cur.execute("SELECT * FROM procedures;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metpro
# met_cur.execute("SELECT * FROM procedures;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in observations

# check data in a table astobs
# ast_cur.execute("SELECT * FROM observations;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covobs
# cov_cur.execute("SELECT * FROM observations;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metobs
# met_cur.execute("SELECT * FROM observations;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

# Copy a table from one database to another
(empty Rehealthco SQLite database schon erstellt)

In [ ]:
# Get connections to the databases 
# von asthma nach rehealthco
def copy_table(rhc_conn, ast_cur, rhc_cur, table_name):
  ast_cur.execute(f'SELECT * FROM {table_name}')
  output = ast_cur.fetchall()   # Returns the results as a list.
  # Insert those contents into another table.
  for row in output:
      rowlen = len(row)
      repstring = ""
      for i in range(rowlen -1):
        repstring += "?,"
      repstring += "?"
      rhc_cur.execute(f'INSERT INTO {table_name} VALUES ({repstring})', row)
  # Cleanup
  rhc_conn.commit()

In [ ]:
# alle Tabellen vereinen in rehealthco
copy_table(rhc_conn, ast_cur, rhc_cur, "patients") # 1035 patienten
copy_table(rhc_conn, cov_cur, rhc_cur, "patients") # nun 2085 patienten
copy_table(rhc_conn, met_cur, rhc_cur, "patients") # nun 3113 patienten

copy_table(rhc_conn, ast_cur, rhc_cur, "conditions") # 260 conditions
copy_table(rhc_conn, cov_cur, rhc_cur, "conditions") # 6635 conditions
copy_table(rhc_conn, met_cur, rhc_cur, "conditions") # 7041 conditions

copy_table(rhc_conn, ast_cur, rhc_cur, "medications") # 505 medications
copy_table(rhc_conn, cov_cur, rhc_cur, "medications") # 1395 medications
copy_table(rhc_conn, met_cur, rhc_cur, "medications") # 1715 medications

copy_table(rhc_conn, ast_cur, rhc_cur, "procedures") # 1715 procedures
copy_table(rhc_conn, cov_cur, rhc_cur, "procedures") # 1715 procedures
copy_table(rhc_conn, met_cur, rhc_cur, "procedures") # 1715 procedures

copy_table(rhc_conn, ast_cur, rhc_cur, "observations") # 31220 observations
copy_table(rhc_conn, cov_cur, rhc_cur, "observations") # 151157 observations
copy_table(rhc_conn, met_cur, rhc_cur, "observations") # 182271 observations

In [ ]:
# Tabelleninhalt rehealthco als pandas dataframe
rhcpat_df = pd.read_sql('''SELECT * FROM patients''', rhc_conn)
rhccon_df = pd.read_sql('''SELECT * FROM conditions''', rhc_conn)
rhcmed_df = pd.read_sql('''SELECT * FROM medications''', rhc_conn)
rhcpro_df = pd.read_sql('''SELECT * FROM procedures''', rhc_conn)
rhcobs_df = pd.read_sql('''SELECT * FROM observations''', rhc_conn)

# print(rhcpat_df)
# print(rhccon_df)
# print(rhcmed_df)
# print(rhcpro_df)
# print(rhcobs_df)

In [ ]:
# Kompelette Tabelle abfragen

# rhc_cur.execute("SELECT * FROM patients;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

# rhc_cur.execute("SELECT * FROM conditions;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

# rhc_cur.execute("SELECT * FROM medications;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

# rhc_cur.execute("SELECT * FROM procedures;")
# rows = rhc_cur.fetchall()
# for row in rows:
# print(row)

# rhc_cur.execute("SELECT * FROM observations;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# 1-Zeiltenanfrage
# rhc_cur.execute("SELECT * FROM patients")
# out = rhc_cur.fetchall()
# print(out)

# Indexe Rehealthco

In [ ]:
# multicolumn Index
rhc_cur.execute("CREATE INDEX idx_contacts_name ON patients (FIRST, LAST)")
rhc_cur.fetchall()

In [ ]:
# show list indexes created for a table
rhc_cur.execute("PRAGMA index_list(patients)")
print(rhc_cur.fetchall())

In [ ]:
rhc_cur.execute("CREATE INDEX idx_patname_conditions ON conditions (PATIENT)")
rhc_cur.execute("PRAGMA index_list(conditions)")
print(rhc_cur.fetchall())

In [ ]:
rhc_cur.execute("CREATE INDEX idx_patname_medications ON medications (PATIENT)")
rhc_cur.execute("PRAGMA index_list(medications)")
print(rhc_cur.fetchall())

In [ ]:
rhc_cur.execute("CREATE INDEX idx_patname_procedures ON procedures (PATIENT)")
rhc_cur.execute("PRAGMA index_list(procedures)")
print(rhc_cur.fetchall())

In [ ]:
rhc_cur.execute("CREATE INDEX idx_patname_observations ON observations (PATIENT)")
rhc_cur.execute("PRAGMA index_list(observations)")
print(rhc_cur.fetchall())

In [ ]:
# get all indexes from a database 
rhc_cur.execute("select type, name, tbl_name, sql FROM sqlite_master WHERE type='index'")
print(rhc_cur.fetchall())

In [ ]:
# # Drop index
# # cur.execute("DROP INDEX idx_contacts_name")
# cur.execute("PRAGMA index_list(patients)")
# print(cur.fetchall())

# Query the tables, join two tables

In [ ]:
# noch nicht sinnvoll für uns, hier beispielhaft inner join asthma patienten tabelle -> inner join -> asthma medication tabelle
# verbindung_astpat_astmed = "SELECT * FROM medications INNER JOIN patients ON medications.PATIENT = patients.Id"
# ast_cur.execute(verbindung_astpat_astmed)
# records = ast_cur.fetchall()
# for row in records:
#   print(row)

In [ ]:
# Left Join
# cov_cur.execute('''SELECT * FROM medications LEFT JOIN patients ON medications.PATIENT = patients.Id''')
# records = cov_cur.fetchall()
# for row in records:
#   print(row)

# Prediction (Supervised Learning)

In [ ]:
# install the newest version,  
# uncomment this line in first runtime, it needs to restart runtime
!pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

# Exploratory Data Analysis (EDA)


In [ ]:
ProfileReport(rhcpat_df) # deathdate: 3000 fehlende Todesdaten, bei insg. 3113 Patienten -> 113 Tote

In [ ]:
# ProfileReport(rhccon_df) # 43 unterschiedliche Descriptions
# ProfileReport(rhcmed_df)
# ProfileReport(rhcpro_df)
# ProfileReport(rhcobs_df) # 71 verschiedene (Vitalzeichen/Laborwerte)